## Importing dependencies

In [130]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd


## Initialize workspace

In [132]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-141702
aml-quickstarts-141702
southcentralus
d4ad7261-832d-46b2-b093-22156001df5b


## Auto ML Experiment

In [131]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'data_quality_analysis'
project_folder = './catalogue-scorecard-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
data_quality_analysis,quick-starts-ws-141702,Link to Azure Machine Learning studio,Link to Documentation


## Create an AmlCompute cluster

In [133]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "comp"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_v2',#for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Data import

In [135]:
import pandas as pd

auto_ds_path="https://raw.githubusercontent.com/shatakshipachori/data_quality_analysis/main/catalogue-scorecard.csv"
data=TabularDatasetFactory.from_delimited_files(path=auto_ds_path)


## AutoML Configuration

In [136]:
import logging
import time

# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations":5,
    "primary_metric":'normalized_root_mean_squared_error',
    "enable_early_stopping":True,
    "experiment_timeout_minutes":30,
    "max_concurrent_iterations":5,
   }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='regression',
    compute_target='comp',
    training_data=data,
    label_column_name='score',
    **automl_settings
    )

## Submit the experiment

In [137]:
automl_run=experiment.submit(config=automl_config,show_output=True)

Running on remote.
No run_configuration provided, running on comp with default configuration
Running on remote compute: comp
Parent Run ID: AutoML_5c98a544-75e6-424d-97da-798a6fafe860

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high car

## Run Details

In [12]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary de

{'runId': 'AutoML_7d5080b1-bb8e-49af-9e32-8ee709929dba',
 'target': 'comp',
 'status': 'Completed',
 'startTimeUtc': '2021-03-31T17:06:13.702633Z',
 'endTimeUtc': '2021-03-31T17:20:18.316458Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'comp',
  'AMLSettingsJsonString': '{"path":null,"name":"data_quality_analysis","subscription_id":"d4ad7261-832d-46b2-b093-22156001df5b","resource_group":"aml-quickstarts-141702","workspace_name":"quick-starts-ws-141702","region":"southcentralus","compute_target":"comp","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"normalized_root_mean_squared_error","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_va

## Best Model

In [138]:
# Retrieve and save your best automl model.
best_automl_run,best_automl_model=automl_run.get_output()
best_run_metrics=automl_run.get_metrics()
for primary_metric in best_run_metrics:
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)

experiment_status ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection']
experiment_status_description ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.']
median_absolute_error 0.000266984689353611
normalized_mean_absolute_error 0.0009452366980462927
root_mean_squared_log_error 0.0005712604223994724
normalized_root_mean_squared_log_error 0.0017902335376288419
spearman_correlation 0.999185767688334
r2_score 0.9999533980442392
root_mean_squared_error 0.0009688033933163006
mean_absolute_error 0.0005009754499645351
explained_variance 0.999953402885974
normalized_root_mean_squared_error 0.0018279309307854727
normalized_median_absolute_error 0.0005037446968936056
mean_absolute_percent

In [139]:
print(best_automl_run)
best_automl_run

Run(Experiment: data_quality_analysis,
Id: AutoML_5c98a544-75e6-424d-97da-798a6fafe860_0,
Type: azureml.scriptrun,
Status: Completed)


Experiment,Id,Type,Status,Details Page,Docs Page
data_quality_analysis,AutoML_5c98a544-75e6-424d-97da-798a6fafe860_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Regression using Linear Regression

In [140]:
data=data.to_pandas_dataframe()

In [141]:
type(data)

pandas.core.frame.DataFrame

In [145]:
data.columns

Index(['_id', 'completeness', 'freshness', 'metadata', 'usability', 'score',
       'score_norm', 'grade', 'grade_norm'],
      dtype='object')

In [148]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data=data.drop(['package','recorded_at','accessibility','version'], axis=1)

#instantiate LabelEncoder
le=LabelEncoder()

#LabelEncode grade column of data
data['grade']=le.fit_transform(data['grade'])
data['grade_norm']=le.fit_transform(data['grade_norm'])

#inspecting encoded df
data.head()

,_id,completeness,freshness,metadata,usability,score,score_norm,grade,grade_norm
0,186,0.69,0.50,0.84,0.86,0.78,0.76,2,2
1,187,1.00,0.00,0.25,0.85,0.54,0.31,0,0
2,188,0.98,1.00,0.25,0.69,0.66,0.52,0,0
3,189,0.96,1.00,0.75,0.94,0.91,1.00,1,1
4,190,0.83,1.00,0.75,0.87,0.87,0.92,1,1


In [91]:
y=data.drop(['score'],axis=1)
X=data

## Import the class

In [92]:
from sklearn.model_selection import train_test_split
import argparse
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

## Model Fitting

In [93]:
from sklearn.linear_model import LinearRegression

#instantiate
linreg=LinearRegression()
linreg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Model Training

In [94]:
print(linreg.intercept_)
print(linreg.coef_)

[-5.45696821e-12 -7.77156117e-16 -7.77156117e-16  7.21644966e-16
  5.55111512e-16  1.33226763e-15  1.55431223e-15]
[[ 1.00000000e+00 -5.33422275e-14  8.29114543e-14 -5.71606415e-14
  -6.96334584e-14 -3.37345643e-12  1.83381172e-12 -2.02720382e-14]
 [ 3.65572781e-21  1.00000000e+00 -4.44089210e-16 -6.66133815e-16
  -1.56472058e-15  3.91353616e-15 -6.10622664e-16 -1.66533454e-16]
 [-4.38070893e-21 -2.22044605e-16  1.00000000e+00 -8.88178420e-16
  -1.22298005e-15  3.89965837e-15  3.60822483e-16 -5.27355937e-16]
 [-3.92386217e-21  0.00000000e+00  2.77555756e-16  1.00000000e+00
   3.83373888e-16 -1.69309011e-15 -1.38777878e-16 -8.32667268e-17]
 [ 1.87055911e-21  5.20417043e-16  2.14238349e-16 -1.26634814e-16
   1.00000000e+00 -1.83793952e-15  4.94829872e-16  1.86482774e-17]
 [-1.26999792e-21  1.11022302e-16  2.22044605e-16 -8.32667268e-17
  -4.82253126e-16 -2.02615702e-15  1.00000000e+00 -2.77555756e-16]
 [ 3.44916434e-21 -1.27675648e-15 -6.93889390e-16 -6.66133815e-16
  -8.63024929e-16  2.

## Model Prediction

In [95]:
y_pred=linreg.predict(X_test)
y_pred

array([[ 1.55560000e+04,  9.40000000e-01,  1.00000000e+00, ...,
         9.50000000e-01,  1.00000000e+00,  1.00000000e+00],
       [ 9.39000000e+03,  6.90000000e-01,  1.00000000e+00, ...,
         7.20000000e-01,  4.70000000e-01,  3.05988240e-16],
       [ 9.59800000e+03,  6.40000000e-01, -4.37113789e-17, ...,
         8.00000000e-01,  2.50000000e-01,  8.19142981e-16],
       ...,
       [ 1.32850000e+04,  7.30000000e-01, -4.36780827e-17, ...,
         8.30000000e-01,  2.40000000e-01,  7.64058383e-16],
       [ 1.12270000e+04,  1.00000000e+00,  1.00000000e+00, ...,
         8.30000000e-01,  8.70000000e-01,  1.00000000e+00],
       [ 4.57600000e+03,  6.40000000e-01, -2.53196834e-17, ...,
         8.00000000e-01,  2.40000000e-01,  8.09176505e-16]])

## Model Accuracy (Root mean squared error)

In [97]:
from sklearn import metrics
print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

1.2624372939949376e-12
